In [ ]:
!pip install konlpy
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661628 sha256=8622ce33d9253535f4703e3cdceab9ae020141bddfdf37b3ac18673a39a94bba
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.9 MB/s eta 0:00:00
     ━━━

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import itertools
import numpy as np
import time, datetime
import matplotlib.pyplot as plt

In [ ]:
# GPU 사용
device = torch.device("cuda:0")

In [ ]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
# 형태소 분석 후 언어정보점수 추가
def add_linguistic_score(sentence, a, b, c, d, e, f, g):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
        if p[1] == 'NNP': # 고유 명사
            score += a
        elif p[1] == 'NNG': # 일반 명사
            score += b
        elif p[1] == 'VV': # 동사 -> 서술어
            score += c
        elif p[1] == 'SL': # 외국어
            score += d
        elif p[1] == 'JKS': # 주격 조사 -> 주어
            score += e
        elif p[1] == 'JKO': # 목적격 조사 -> 목적어
            score += f
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words: # 따옴표 안의 단어
        score += g
    return score

# 문장 어절 단위로 토큰화
def word_tokenizer(sentence):
    return sentence.split(' ')

# 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
        if "'" in token:
            if in_quote:
                quoted_word += " " + token
                processed_tokens.append(quoted_word)
                quoted_word = ""
                in_quote = False
            else:
                quoted_word = token
                in_quote = True
        else:
            if in_quote:
                quoted_word += " " + token
            else:
                processed_tokens.append(token)
    return processed_tokens

# 압축 문장 생성
def compress_sentence(sentence, a, b, c, d, e, f, g):

    token = word_tokenizer(sentence)
    token = process_quoted_words(token)

    # 압축 문장 후보군 생성 및 언어정보점수 계산
    max_n = len(token) - 4
    # max_n = 4
    compressed_candidates = []
    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            compressed_tokens = token[:i] + token[i + n:]
            compressed_sentence = " ".join(compressed_tokens)
            # 형태소 점수 계산
            score = add_linguistic_score(compressed_sentence, a, b, c, d, e, f, g)
            # 각 요약 문장 후보에 형태소 점수 추가
            compressed_candidates.append((compressed_sentence, score, n))

    # perplexity 계산
    def calculate_perplexity_score(sentence):

        # KoBERT 모델이 이해할 수 있는 형태로 tokenize
        tokens = tokenizer.tokenize(sentence)
        token_ids = tokenizer.convert_tokens_to_ids(tokens) # tokenized word -> 정수 인덱스

        # [MASK] 토큰 위치 찾기
        mask_token_index = token_ids.index(tokenizer.mask_token_id)

        # 입력값과 출력값 생성
        input_ids = torch.tensor([token_ids]).to(device)
        outputs = model(input_ids) # 예측값
        predictions = outputs[0]

        # [MASK] 토큰 위치에 대한 예측값 추출
        masked_predictions = predictions[0, mask_token_index]

        # softmax 함수를 이용하여 확률값을 확률 분포로 변환
        probs = torch.softmax(masked_predictions, dim=-1)

        # perplexity 계산
        perplexity = torch.exp(torch.mean(torch.log(probs)))

        return perplexity.item()

    perplexity_scores = []
    compressed_candidates_with_score = []

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            mask_idx = list(range(i, i + n))
            masked_tokens = list(token)
            for j in mask_idx:
                masked_tokens[j] = "[MASK]"
            masked_sentence = " ".join(masked_tokens)

            perplexity_score = calculate_perplexity_score(masked_sentence) # perplexity 계산
            linguistic_score = compressed_candidates[i][1] # 형태소 점수 가져오기
            final_score = 1/perplexity_score * linguistic_score #* (-1) # 최종 점수 계산

            # 최종 점수 저장
            perplexity_scores.append(perplexity_score)
            compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

    # 최종 요약 문장 선택 (최종 점수가 가장 낮은)
    compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1], reverse = True)

    if compressed_candidates_with_score_sorted:
      final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
      selected_n = compressed_candidates_with_score_sorted[0][2]
    else:
      # 리스트가 비어있을 경우에 대한 예외 처리 또는 기본값 설정
      final_compressed_sentence = "No compressed sentence available"
      selected_n = 0

    return final_compressed_sentence # 최종 요약 문장 반환

# 코사인 유사도 계산을 위한 모델 임베딩 함수 정의
def encode_sentence(sentence):

    # 문장을 토큰화하고 토큰을 ID로 변환
    tokens = tokenizer.tokenize(sentence)

    if len(tokens) == 0 :
        return None

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # 모델 입력으로 사용할 수 있도록 텐서로 변환하고 GPU로 이동
    input_ids = torch.tensor([token_ids]).to(device)

    # 모델에 입력을 전달하고 출력을 얻음
    with torch.no_grad():
        outputs = model(input_ids)

    # 모델의 마지막 hidden state에서 평균을 계산하여 문장 임베딩 얻음
    hidden_states = outputs[0]
    sentence_embedding = torch.mean(hidden_states, dim=1).squeeze().cpu().numpy()

    sentence_embedding = sentence_embedding.reshape(1,-1)

    return sentence_embedding

In [ ]:
import re

def compress_sentence(sentence):

  # 문장 어절 단위로 토큰화
  def word_tokenizer(sentence):
    return sentence.split(' ')

  token = word_tokenizer(sentence)

  # 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
  def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
      if "'" in token:
        if in_quote:
          quoted_word += " " + token
          processed_tokens.append(quoted_word)
          quoted_word = ""
          in_quote = False
        else :
          quoted_word = token
          in_quote = True
      else :
        if in_quote:
          quoted_word += " " + token
        else:
          processed_tokens.append(token)
    return processed_tokens
  token = process_quoted_words(token)

  # 형태소 분석 후 언어정보점수 추가
  def add_linguistic_score(sentence):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
      if p[1] == 'NNP': # 고유명사
        score += 0.0001
      elif p[1] == 'NNG': # 일반명사
        score += 0.0001
      elif p[1] == 'vv': # 동사 -> 서술어
        score += 0.0001
      elif p[1] == 'SL' : # 외국어
        score += 0.0001
      elif p[1] == 'JKS' : # 주격 조사 -> 주어
        score += 0.0001
      elif p[1] == 'JKO' : # 목적격 조사 -> 목적어
        score += 0.0001
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words: # 따옴표 안의 단어들
      score += 0.0001
    return score

  # 압축 문장 후보군 생성 및 언어정보점수 계산
  # n-gram
  compressed_candidates = []
  max_n = len(token) - 4 # 문장의 주성분인 주어,목적어,보어,서술어는 최소한 남겨두기 위해
  for n in range(1, max_n + 1):
    for i in range(len(token) - n + 1):
      compressed_tokens = token[:i] + token[i+n:]
      compressed_sentence = " ".join(compressed_tokens)
      # 언어정보점수 계산
      score = add_linguistic_score(compressed_sentence)
      # 압축 문장 후보에 언어정보점수 추가
      compressed_candidates.append((compressed_sentence,score,n))

  # perplexity 계산
  def calculate_perplexity_score(sentence):

    # KoBERT 모델이 이해할 수 있는 형태로 tokenize
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens) # tokenized word -> 정수 인덱스

    # [MASK] 토큰 위치 찾기
    mask_token_index = token_ids.index(tokenizer.mask_token_id)

    # 입려값과 출력값 생성
    input_ids = torch.tensor([token_ids])
    outputs = model(input_ids) # 예측값
    predictions = outputs[0]

    # [MASK] 토큰 위치에 대한 예측값 추출
    masked_predictions = predictions[0, mask_token_index]

    # softmax 함수를 이용하여 확률값을 확률 분포로 변환
    probs = torch.softmax(masked_predictions, dim = -1)

    # perplexity 계산
    perplexity = torch.exp(torch.mean(torch.log(probs)))

    return perplexity.item()

  perplexity_scores = []
  compressed_candidates_with_score = []

  # perplexity와 linguistic score를 포함한 정보를 저장
  for n in range(1, max_n + 1):
      for i in range(len(token) - n + 1):
          mask_idx = list(range(i, i + n))
          masked_tokens = list(token)
          for j in mask_idx:
              masked_tokens[j] = "[MASK]"
          masked_sentence = " ".join(masked_tokens)

          # perplexity score 계산
          perplexity_score = calculate_perplexity_score(masked_sentence)

          # 언어정보점수 가져오기
          linguistic_score = add_linguistic_score(masked_sentence)

          # 최종 점수 = perplexity_score + 언어정보점수
          final_score = 1 / perplexity_score * linguistic_score

          # perplexity_score, 언어정보점수, 최종 점수 저장
          perplexity_scores.append(perplexity_score)
          compressed_candidates_with_score.append((
              re.sub(r'\[MASK\]\s*', '', masked_sentence),
              final_score,
              n,
              perplexity_score,  # perplexity score 추가
              linguistic_score  # 언어정보점수 추가
          ))

  # 결과를 DataFrame으로 변환
  df = pd.DataFrame(compressed_candidates_with_score, columns=['압축 문장 후보', '최종 점수', 'n-gram', 'Perplexity Score', 'Linguistic Score'])
  df['n-gram'] = df['n-gram'].astype(int)
  pd.options.display.float_format = '{:.15f}'.format

  # DataFrame를 '최종 점수'를 기준으로 내림차순 정렬
  df_sorted = df.sort_values(by='최종 점수', ascending=False)

  # 표로 DataFrame 출력
  display(df_sorted)

  # 최종 압축 문장 선택
  compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1], reverse=True)

  final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
  selected_n = compressed_candidates_with_score_sorted[0][2]

  print(f"n-gram: {selected_n}")
  print(f"입력 문장: {sentence}")
  print(f"최종 압축 문장: {final_compressed_sentence}")

  return final_compressed_sentence

In [ ]:
input_sentence = input()

만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 중단하거나 다른 약물로 대체해야 한다


In [ ]:
compress_sentence(input_sentence)

,압축 문장 후보,최종 점수,n-gram,Perplexity Score,Linguistic Score
69,만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을,49.715883300534550,5,0.000032182873838,0.001600000000000
43,만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 ...,49.409525836062969,3,0.000030358518416,0.001500000000000
44,만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 ...,49.222480803032610,3,0.000030473880543,0.001500000000000
114,만약 지방간의 원인이 되는 약물을 복용하고,49.196367282725909,10,0.000032522726542,0.001600000000000
120,만약 지방간의 원인이 되는 약물을,49.077480460484303,11,0.000032601510611,0.001600000000000
...,...,...,...,...,...
17,만약 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 중단하거나 다른 ...,37.717478900199666,2,0.000034466778743,0.001300000000000
26,만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 다른 약...,37.696764721845213,2,0.000034485718061,0.001300000000000
10,만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 중단하거...,37.580858821374100,1,0.000034592078009,0.001300000000000
4,만약 지방간의 원인이 되는 복용하고 있다면 주치의와 상의하여 약물의 복용을 중단하거...,37.534618511158051,1,0.000034634693293,0.001300000000000


n-gram: 5
입력 문장: 만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 중단하거나 다른 약물로 대체해야 한다
최종 압축 문장: 만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 


'만약 지방간의 원인이 되는 약물을 복용하고 있다면 주치의와 상의하여 약물의 복용을 '